In [1]:
#!pip install tflearn
import numpy as np
#import tflearn
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk 
from nltk import LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
#nltk.download('stopwords')


Initialize stemmer, stopwords and clean sentence method

In [2]:
stemmer = SnowballStemmer("english")
#stemmer = LancasterStemmer()

stopwords = set(stopwords.words('english'))
#print(stopwords)

In [3]:
import re
only_alnum = re.compile(r"[^\w]+") ## \w => unicode alphabet

def clean_sentence(sentence):
    sentence = re.sub(only_alnum, " ", sentence).strip()
    words = nltk.word_tokenize(sentence)
    #words = [stemmer.stem(w.lower()) for w in words if w not in stopwords]
    words = [w for w in words if w not in stopwords]
    return words


Get all the documents from the intents.json, clean the sentences and add then to a list of documents, along with their respective tags

In [4]:
#import the intent file
import json
intents = []
with open('intents.json') as json_data:
    intents = json.load(json_data)

classes = []
dictionary = []
documents = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #for each pattern, get the list of words
        words = clean_sentence(pattern)
        tag = intent['tag']
        #get the unique set of classes
        if tag not in classes:
            classes.append(tag)
        #tag each document with its class
        documents.append((words, tag))
        #unique set of stemmed words
        dictionary.extend(words)

dictionary = set(dictionary)
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(dictionary), "unique stemmed words", dictionary)
print(documents)

27 documents
9 classes ['greeting', 'goodbye', 'thanks', 'hours', 'mopeds', 'payments', 'opentoday', 'rental', 'today']
37 unique stemmed words {'hours', 'open', 'take', 'Good', 'mopeds', 'Thank', 'Thanks', 'work', 'How', 'rent', 'helpful', 'day', 'Hello', 'like', 'accept', 'See', 'When', 'cards', 'What', 'cash', 'That', 'today', 'Mastercard', 'I', 'Bye', 'Are', 'kinds', 'credit', 'Hi', 'Do', 'Is', 'Which', 'moped', 'anyone', 'Goodbye', 'later', 'Can'}
[(['Hi'], 'greeting'), (['How'], 'greeting'), (['Is', 'anyone'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank'], 'thanks'), (['That', 'helpful'], 'thanks'), (['What', 'hours', 'open'], 'hours'), (['What', 'hours'], 'hours'), (['When', 'open'], 'hours'), (['Which', 'mopeds'], 'mopeds'), (['What', 'kinds', 'mopeds'], 'mopeds'), (['What', 'rent'], 'mopeds'), (['Do', 'take', 'credit', 'cards'], 'payments'), 

Use googles precomputed word2vec mapping

In [5]:
#use googles word vector model
import gensim
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models import Phrases
import logging

#use the existing google bin to get word2vec representation
model = gensim.models.KeyedVectors.load_word2vec_format('../Datasets/GoogleNews-vectors-negative300.bin', binary=True, limit=500000)

In [6]:
#create own word 2 vec
#perform word2vec on these words
num_features = 300    # Word vector dimensionality                      
min_word_count = 1   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

#build a word2vec representation using the given docs
#wp = word2vec.Word2Vec(docs, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)



Convert all the words into word2vec representation
compute sentence2vec = summation of word2vec representations of all the words in that sentence

In [7]:
#converting documents into word2vec representations
def convert_sentence2vec(documents, isUseZeros, isUseOnes):
    missing_word_vecs={}
    number_of_docs = len(documents)
    word2vec_rep = np.zeros((number_of_docs, num_features))
    count = 0 
    labels = []
    i=0
    for document in documents:
        tag = document[1]
        doc_words = document[0]
        #print(doc_words)
        for word in doc_words: 
            try:
                word2vec_rep[i]+=model[word]
            except:
                '''The word isn't in our pretrained word-vectors, hence we add a random gaussian noise
                    to account for this. We store the random vector we assigned to the word, and reuse 
                    the same vector during test time to ensure consistency.'''
                if word  not in missing_word_vecs.keys():
                    if isUseZeros:
                        missing_word_vecs[word] = np.zeros((num_features))
                    elif isUseOnes:
                        missing_word_vecs[word] = np.ones((num_features))
                    else:
                        missing_word_vecs[word] = np.random.normal(-0.25, 0.25, num_features)
                word2vec_rep[i]+=missing_word_vecs[word]
                count +=1
        labels.append(tag)
        i+=1
    return word2vec_rep, labels, missing_word_vecs

sent2vec_rep, labels, missing_word_vecs = convert_sentence2vec(documents, True, False)

In [12]:
print(sent2vec_rep.shape)
print(labels)
#print(model['I']+ model['like']+model['rent']+model['moped'])
#print(sent2vec_rep[-3])

(27, 300)
['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'hours', 'hours', 'hours', 'mopeds', 'mopeds', 'mopeds', 'payments', 'payments', 'payments', 'opentoday', 'opentoday', 'opentoday', 'rental', 'rental', 'rental', 'today']


Train the word2vec_rep

In [ ]:
# train this vectorial representation (multi class classification problem)
